In [199]:
import pandas as pd
import numpy as np

results = pd.read_csv("results/PRSH_results.csv")
results = results.rename(columns={"env":"e","mutation":"m","elapse":"v"})

In [200]:
env = 1

subset = results[(results["e"]==env)]
groups = subset.groupby(["k","v","m"])
groups["profit"].mean().sort_values(ascending=False)

k   v    m
6   128  2    1274.50500
14  128  1    1274.11675
2   256  1    1272.42875
4   256  0    1266.09450
         1    1265.44775
                 ...    
12  128  1    1173.70175
2   32   1    1164.38675
8   32   1    1158.77300
14  64   1    1155.30375
12  32   1    1136.87450
Name: profit, Length: 96, dtype: float64

In [201]:
import numpy as np
from scipy import stats

def ksTest(s):
	return stats.kstest(s, 'norm', (s.mean(), s.std())).pvalue

ksresults = groups["profit"].apply(ksTest)

In [202]:
source = ""

for k in np.unique(results["k"]):
    source += str(k) + " & "
    for m in np.unique(results["m"]):
        for i,v in enumerate(np.unique(results["v"])):
            kv = (ksresults[(k,v,m)]).round(2)
            source += str(kv)
            if (i < len(np.unique(results["v"]))-1) or (m<2):
                source += " & "
            else:
                source += " \\\ \hline "
print(source)

2 & 0.97 & 0.82 & 0.24 & 0.86 & 0.38 & 0.32 & 0.88 & 0.97 & 0.98 & 0.78 & 0.8 & 0.74 \\ \hline 4 & 0.37 & 0.89 & 0.76 & 0.91 & 0.99 & 0.99 & 0.91 & 0.97 & 0.62 & 0.85 & 0.98 & 0.87 \\ \hline 6 & 0.83 & 0.94 & 1.0 & 0.53 & 0.88 & 0.92 & 0.58 & 0.98 & 0.82 & 0.78 & 0.63 & 0.99 \\ \hline 8 & 0.61 & 0.27 & 0.51 & 0.97 & 0.32 & 0.7 & 0.96 & 0.7 & 0.94 & 0.82 & 0.67 & 0.24 \\ \hline 10 & 0.97 & 0.99 & 0.94 & 0.99 & 0.54 & 0.94 & 0.74 & 0.72 & 0.14 & 0.51 & 0.95 & 0.99 \\ \hline 12 & 0.99 & 0.74 & 0.17 & 0.63 & 0.45 & 0.95 & 0.97 & 0.51 & 0.99 & 0.99 & 0.99 & 0.73 \\ \hline 14 & 0.52 & 0.88 & 0.52 & 0.67 & 0.89 & 0.99 & 0.24 & 0.99 & 0.5 & 0.71 & 0.91 & 0.99 \\ \hline 16 & 1.0 & 0.83 & 0.51 & 0.75 & 0.96 & 0.68 & 0.93 & 0.95 & 0.99 & 0.77 & 0.32 & 0.98 \\ \hline 


In [203]:
import numpy as np
from scipy.stats import norm

bestk = 6
bestv = 128
bestm = 2

def doublezTest(y):
    x = results[(results["m"]==bestm)&(results["k"]==bestk)&(results["v"]==bestv)]["profit"]
    varx = x.var()*(x.shape[0]-1)/x.shape[0] 
    vary = y.var()*(y.shape[0]-1)/y.shape[0] 
    n1 = x.shape[0]
    n2 = y.shape[0]
    z = (x.mean()-y.mean())/np.sqrt(varx/n1+vary/n2)
    p = 1 - norm.cdf(abs(z))
    return p

zTestResult = groups["profit"].apply(doublezTest)

In [204]:
source = ""

paralist = []

for k in np.unique(results["k"]):
    source += str(k) + " & "
    for m in np.unique(results["m"]):
        for i,v in enumerate(np.unique(results["v"])):
            kv = (results[(results["k"]==k)&(results["m"]==m)&(results["e"]==env)&(results["v"]==v)].mean()["profit"]/1000).round(2)
            if (k, v, m) == (bestk, bestv, bestm):
                source += "\\textbf{%.2f}" % np.log10(kv)
                paralist.append((m, k, v))
            elif zTestResult[(k,v,m)]>0.05:
                source += "\\underline{%.2f}" % np.log10(kv) + "_{%.2f}" % zTestResult[(k,v,m)].round(2)
                paralist.append((m, k, v))
            else:
                source += str(np.log10(kv).round(2))
            if (i < len(np.unique(results["v"]))-1) or (m<2):
                source += " & "
            else:
                source += " \\\ \hline "
print(source)

2 & 0.07 & \underline{0.09}_{0.09} & 0.08 & \underline{0.09}_{0.08} & 0.06 & \underline{0.09}_{0.06} & 0.08 & \underline{0.10}_{0.46} & \underline{0.09}_{0.11} & 0.07 & 0.08 & \underline{0.09}_{0.06} \\ \hline 4 & \underline{0.09}_{0.11} & 0.08 & \underline{0.10}_{0.19} & \underline{0.10}_{0.38} & 0.07 & 0.08 & 0.09 & \underline{0.10}_{0.37} & 0.09 & 0.08 & \underline{0.10}_{0.31} & 0.08 \\ \hline 6 & \underline{0.09}_{0.10} & 0.08 & 0.09 & 0.09 & 0.08 & 0.09 & \underline{0.10}_{0.37} & \underline{0.09}_{0.09} & \underline{0.09}_{0.10} & \underline{0.10}_{0.16} & \textbf{0.10} & \underline{0.09}_{0.10} \\ \hline 8 & \underline{0.10}_{0.21} & \underline{0.10}_{0.30} & 0.08 & 0.08 & 0.06 & 0.08 & 0.08 & 0.08 & 0.08 & \underline{0.10}_{0.23} & 0.09 & 0.08 \\ \hline 10 & \underline{0.09}_{0.12} & \underline{0.09}_{0.05} & 0.08 & \underline{0.09}_{0.13} & \underline{0.10}_{0.18} & 0.08 & \underline{0.09}_{0.06} & 0.09 & 0.07 & 0.09 & \underline{0.09}_{0.17} & \underline{0.09}_{0.15} \\ \hli

In [205]:
print(len(paralist),paralist)

37 [(0, 2, 64), (0, 2, 256), (1, 2, 64), (1, 2, 256), (2, 2, 32), (2, 2, 256), (0, 4, 32), (0, 4, 128), (0, 4, 256), (1, 4, 256), (2, 4, 128), (0, 6, 32), (1, 6, 128), (1, 6, 256), (2, 6, 32), (2, 6, 64), (2, 6, 128), (2, 6, 256), (0, 8, 32), (0, 8, 64), (2, 8, 64), (0, 10, 32), (0, 10, 64), (0, 10, 256), (1, 10, 32), (1, 10, 128), (2, 10, 128), (2, 10, 256), (0, 12, 64), (0, 12, 256), (1, 14, 32), (1, 14, 128), (1, 14, 256), (2, 14, 128), (2, 14, 256), (0, 16, 256), (1, 16, 256)]
